In [6]:
#本代码（1）把尤同学的调用代码从matlab转换为python代码. （2）MUST toolbox里的函数则使用matlab.engine调用（不修改为python代码）
import matlab
import matlab.engine
import os
import numpy as np
import cv2


eng=matlab.engine.start_matlab()   #启动matlab引擎

filepath=os.getcwd()
               
n_pw =np.arange(1,26)
Na = n_pw
alpha_max =np.deg2rad(16)  #% 读取角度值,角度值转换为弧度值可以使用函数：deg2rad.  这里将16度的角度值，换为弧度值，约0.25。    alpha_max = atan(1/2/F_number);
alpha=np.linspace(-alpha_max,alpha_max,len(n_pw))    #% vector of angles [rad]

#%for i=33:68
for i in range(1,3):
    picname = f"D:/matlab/cleanimg/{i:05d}.jpg"  # %读取第33到68张图片
    I = cv2.imread(picname,cv2.IMREAD_GRAYSCALE)
    nl,nc = I.shape  # %返回矩阵I的行、列尺寸信息
   
    toolbox_path="C:/Program Files/MATLAB/R2023a/toolbox/MUST" #添加工具箱路径
    eng.addpath(toolbox_path)
    
    # % Choose a cardiac phased array
    param = eng.getparam('L11-5v');   # %选择不同的 transducer参数， L11-5v' (128-element, 7.6-MHz linear array)
    param['fc'] = int(np.random.randint(50,131)*1e5)  #%随机生成50到130之间的一个数。随机生成参数fc、fs、kerf、width的值。
    param['fs'] = 4*param['fc']
    param['kerf'] = np.random.randint(2,6)*1e-5
    param['width'] =np.random.randint(25,31)*1e-5;
    param['pitch'] = param['kerf'] + param['width']
    array_lenth = param['Nelements']*param['pitch']   #%计算array_lenth
    
    #% Create a 2-D distribution of scatterers whose depth is 15 cm. RC will contain the reflection coefficients.
    xs,_,zs,RC = eng.genscat(float('nan'),5e-2,param,matlab.double(I.tolist()))  # %Generate a distribution of scatterers，使用genscat函数生成散射体在深度为5cm处的分布以及反射系数RC。特殊字符"NaN"表示Not a NUmber既不是实数也不是复数的“非数字”的值，一个没有意义的数字
    scale = (np.max(xs)-np.min(xs))/array_lenth   #%计算缩放比例scale，然后下面将xs和zs按照该比例进行缩放
    xs/=scale
    zs=20/1000 + zs/scale
    #% Create a 256x256 polar grid with IMPOLGRID.创建一个256x256的极坐标网格。
    xi = linspace(np.min(xs),np.max(xs),nc)  #% in m
    zi = linspace(np.min(zs),np.max(zs),nl)  #% in m
    xi,zi = np.meshgrid(xi,zi)

    IQc = np.zeros_like(xi,dtype=complex)  #% will contain the compound I/Q
    opt=eng.struct()
    opt.ElementSplitting = 1  #% to make simulations faster 来加快模拟速度
    opt.WaitBar = false  #% no progress bar for SIMUS， SIMUS没有进度条
    param.fs = 4*param.fc  #% sampling frequency
    param.fnumber = []

    #h = waitbar(0,'SIMUS & DAS...');   #%更新进度条。第一个参数必须为0到1之间的数，第二个参数message为显示的信息
    for k in range(len(n_pw)):
        dels = eng.txdelay(param,alpha[k]) #% transmit delays
        RF = eng.simus(xs,zs,RC,dels,param,opt) #% RF simulation
        IQ = eng.rf2iq(RF,param)  #% I/Q （射频信号）， demodulation 解调
        IQb = eng.das(IQ,xi.tolist(),zi.tolist(),dels,param)  #% DAS beamforming
        IQc += np.array(IQb._data).reshape((nl,nc)) #% compounding
        #waitbar(k/length(alpha),h,['SIMUS & DAS: ' int2str(k) ' of ' int2str(length(n_pw)) ' completed'])    #%显示进度条
       

    B = eng.bmode(IQc.tolist(),100)   #% log-compressed image
    B=np.array(B._data).reshape((nl,nc))

    #%cd('D:\MUST (1)\US VALIDATE(1)')          %把当前工作目录切换到指定文件夹
    image_path=f"D:/matlab/usimg/{i:05d}.jpg"
    cv2.imwrite(image_path,B)
    
    save_path=f"D:/matlab/usimg/{i:05d}.m"
    np.save(save_path,param)
eng.quit() 
os.chdir(filepath)

ModuleNotFoundError: No module named 'matlab.engine'; 'matlab' is not a package